In [1]:
import torch
import torch.nn as nn 
from torch.nn import functional as F

In [2]:
# Get Data 
import urllib.request
txt_url = "https://s3.amazonaws.com/text-datasets/nietzsche.txt"
urllib.request.urlretrieve(txt_url, 'train.txt')

('train.txt', <http.client.HTTPMessage at 0x120203b90>)

In [3]:
numOfEmbedVariables = 32
batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

In [4]:
# Tokenize
class Tokenizer():
    def __init__(self, characters_list:list) -> None:
        self.characters_list = characters_list

    # map string to int
    def mapStringToInt(self):
        self.MapStrToInt = {ch:i for i, ch in enumerate(self.characters_list)}
        return self.MapStrToInt

    # map to int to string
    def mapIntToString(self):
        self.MapIntToStr = {i:ch for i, ch in enumerate(self.characters_list)}
        return self.MapIntToStr 

    # string to int
    def encoder(self, string:str):
        CharMaps=self.MapStrToInt
        return [CharMaps[c] for c in string]

    # int to string
    def decoder(self, int_list:list):
        CharMaps=self.MapIntToStr
        return ''.join([CharMaps[i] for i in int_list])


In [5]:
# Training and validate 
class DataLoader():
    def __init__(self, data, block_size, batch_size) -> None:
        self.data = data
        self.block_size = block_size
        self.batch_size = batch_size
        
    # split data into validation and training data.
    def splitData(self, splitPercentage=0.9):
        n = int(splitPercentage*len(self.data))
        self.val_data = self.data[n:]
        self.train_data = self.data[:n]

        return self.train_data, self.val_data
    
    # Get chunk of data
    def getBlock(self, start, end, data_list:list):
        return data_list[start:end]

    # get batch 
    def get_batch(self, split):
        # choose train_data or val_data 
        data = self.train_data if split == 'train' else self.val_data
        # generate batch size random numbers 
        ix = torch.randint(len(data) - self.block_size, (self.batch_size,))
        # first block size words 
        # stack to create new dimension 
        x = torch.stack([self.getBlock(start=i, end=i+self.block_size, data_list=self.data) for i in ix])
        # offset of x 
        y = torch.stack([self.getBlock(start=i+1, end=i+self.block_size+1, data_list=self.data) for i in ix])
        return x, y

In [6]:
with open("train.txt", "r", encoding='utf-8') as f:
    text = f.read()

print(f"Len(train.txt): {len(text)}")

Len(train.txt): 600893


In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)
print(chars)


 !"'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyzÆäæéë
84
['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Æ', 'ä', 'æ', 'é', 'ë']


In [8]:
tokenizer = Tokenizer(characters_list = chars)

MapStrToInt = tokenizer.mapStringToInt()
MapIntToStr = tokenizer.mapIntToString()

print(tokenizer.encoder(string="hi there"))
print(tokenizer.decoder(int_list = tokenizer.encoder(string="hi there")))

[60, 61, 1, 72, 60, 57, 70, 57]
hi there


In [9]:
# Encode the train.txt and wrap it in tensor
data = torch.tensor(tokenizer.encoder(text), dtype=torch.long)
print(f"data.shape: {data.shape}")

data.shape: torch.Size([600893])


In [10]:
dataLoader = DataLoader(data = data, 
                        block_size=8,
                        batch_size=4)

In [11]:
# Split the data 
train_data, val_data = dataLoader.splitData()

In [12]:
# Split the data to chunk
# maximum context length for predictions
block_size = 8

x = dataLoader.getBlock(data_list=train_data, start=0, end=block_size)
y = dataLoader.getBlock(data_list=train_data, start=1, end=block_size+1)

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target is {target}")

When input is tensor([39]) the target is 41
When input is tensor([39, 41]) the target is 28
When input is tensor([39, 41, 28]) the target is 29
When input is tensor([39, 41, 28, 29]) the target is 24
When input is tensor([39, 41, 28, 29, 24]) the target is 26
When input is tensor([39, 41, 28, 29, 24, 26]) the target is 28
When input is tensor([39, 41, 28, 29, 24, 26, 28]) the target is 0
When input is tensor([39, 41, 28, 29, 24, 26, 28,  0]) the target is 0


In [13]:
# generate random numbers and seed it so the numbers wont change every time we execute
torch.manual_seed(1337)

# number of independent sequences for parallel process
batch_size = 4

xb, yb = dataLoader.get_batch(split='train')
print("inputs:")
print(xb.shape)
print(xb)

print("targets:")
print(yb.shape)
print(yb)

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b:t+1]
        target = yb[b,t]
        print(f"When input is {context.tolist()} the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[67, 65,  0, 53, 54, 71, 67, 64],
        [ 1,  1, 32,  1, 53, 65,  7,  1],
        [53, 64,  1, 72, 60, 53, 72,  1],
        [61, 67, 66,  1, 53, 66, 56,  1]])
targets:
torch.Size([4, 8])
tensor([[65,  0, 53, 54, 71, 67, 64, 73],
        [ 1, 32,  1, 53, 65,  7,  1, 72],
        [64,  1, 72, 60, 53, 72,  1, 61],
        [67, 66,  1, 53, 66, 56,  1, 57]])
When input is [[67, 65, 0, 53, 54, 71, 67, 64]] the target is 65
When input is [[67, 65, 0, 53, 54, 71, 67, 64], [1, 1, 32, 1, 53, 65, 7, 1]] the target is 0
When input is [[67, 65, 0, 53, 54, 71, 67, 64], [1, 1, 32, 1, 53, 65, 7, 1], [53, 64, 1, 72, 60, 53, 72, 1]] the target is 53
When input is [[67, 65, 0, 53, 54, 71, 67, 64], [1, 1, 32, 1, 53, 65, 7, 1], [53, 64, 1, 72, 60, 53, 72, 1], [61, 67, 66, 1, 53, 66, 56, 1]] the target is 54
When input is [[67, 65, 0, 53, 54, 71, 67, 64], [1, 1, 32, 1, 53, 65, 7, 1], [53, 64, 1, 72, 60, 53, 72, 1], [61, 67, 66, 1, 53, 66, 56, 1]] the target is 71
When in

In [14]:
print(xb)

tensor([[67, 65,  0, 53, 54, 71, 67, 64],
        [ 1,  1, 32,  1, 53, 65,  7,  1],
        [53, 64,  1, 72, 60, 53, 72,  1],
        [61, 67, 66,  1, 53, 66, 56,  1]])


for above and below 

for the first one (67), BigramLangModel will look for the 67th row of the Embedding table

```
tensor([[67, 65,  0, 53, 54, 71, 67, 64],  
        [ 1,  1, 32,  1, 53, 65,  7,  1],  
        [53, 64,  1, 72, 60, 53, 72,  1],  
        [61, 67, 66,  1, 53, 66, 56,  1]])   
```

Attention is a communication mechanism. 

In [15]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(numOfEmbedVariables, head_size, bias=False)
        self.query = nn.Linear(numOfEmbedVariables, head_size, bias=False)
        self.value = nn.Linear(numOfEmbedVariables, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B,T,C = x.shape
        torch.manual_seed(1337)
        x = torch.randn(B,T,C) # private information to the token 
        
        # Single Head 
        k = self.key(x) # (B, T, head_size)
        q = self.query(x) # (B, T, head_size)

        # compute attention scores # Scale Dot-Product Attention 
        weight = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, head_size) @ (B, head_size, T) --> (B, T, T) 
        weight = weight.masked_fill(self.tril[:T,:T] == 0, float('-inf'))  # (B, T, T) # setting it to infinity, we will not aggregate the tokens from the past, in other words future cannot communicate from the past
        weight = F.softmax(weight, dim = -1) # Normalize weight

        # perform te weighted aggregation of the value 
        v = self.value(x) # Thing that get aggregated for the purposes of the single head
        xBagOfWords = weight @ v

        print(xBagOfWords.shape)


In [16]:
class MultiHeadAttention(nn.Module):
    # multi heads of self attention in parallel
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim=-1)

In [17]:
class FeedFoward(nn.Module):
    # a simple linear layer followed by a non-linearity

    def __init__(self, numOfEmbedVariables):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(numOfEmbedVariables, numOfEmbedVariables),
            nn.ReLU(),
            nn.Linear(numOfEmbedVariables, numOfEmbedVariables),
        )

    def forward(self, x):
        return self.net(x)

## Blocks

https://arxiv.org/abs/1512.03385

https://paperswithcode.com/method/residual-block

https://towardsdatascience.com/residual-blocks-building-blocks-of-resnet-fd90ca15d6ec



In [19]:
class Block(nn.Module):
    # Transformer block:  communication followed by computation 
    def __init__(self, n_embed, n_head):
        #  n_embed : embedding demensions,  
        #  n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embed)
        
        def forward(self, x):
            out = torch.cat([h(x) for h in self.heads], dims=-1)
            out = self.proj(out)
            
            return out
        

In [ ]:
class BigramLangModel(nn.Module):
    def __init__(self, vocab_size, numOfEmbedVariables) -> None:
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, numOfEmbedVariables)
        self.position_embedding_table = nn.Embedding(block_size, numOfEmbedVariables) # Embedding table for position of the token 
        self.SelfAttentionHead = MultiHeadAttention(4, numOfEmbedVariables//4) # i.e 4 heads of 8 dimensional self attention 
        self.ffwd = FeedFoward(numOfEmbedVariables)
        self.lm_head = nn.Linear(numOfEmbedVariables, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both tensor 
        tok_emb = self.token_embedding_table(idx) # (Batch(batch size), Time(block size), Channel(embed))
        pos_emb = self.position_embedding_table(torch.arange(T,device=device)) # (T,C) integers of 0 ~ T-1
        x = tok_emb + pos_emb # (B,T,C)
        x = self.SelfAttentionHead(x) # apply one head of self-attention (B,T,C)
        x = self.ffwd(x) # all the token do the FeedForward independently 
        logits = self.lm_head(x) # (Batch(batch size), Time(block size), vocab size)

        if targets is None:
            loss = None
        else:    
            B, T, C = logits.shape # (Batch(batch size), Time(block size), Channel(embed))
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # targets = targets(-1)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in current context 
        for _ in range(max_new_tokens):
            # crop idx to the last block size tokens # we can never have more than block size 
            idx_cond = idx[:,-block_size:]
            # get the predictions 
            logits, loss = self(idx_cond)
            # focus only on the last step 
            logits = logits[:, -1, :]
            # apply softmax to the probabilities 
            probs = F.softmax(logits, dim =-1) #(B,C)
            # sample from distribution 
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            #append sampled index to running sequence
            idx = torch.cat((idx, idx_next), dim = 1)
        
        return idx


: 